In [13]:
# import libraries

import pandas as pd 

### Table 1

In [14]:
# Create Pandas dataframe from JSON file
business = pd.read_json('./../data/yelp_business.json', lines=True)
business.head(3) 

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."


### Table 2

In [15]:
# Create Pandas dataframe from CSV file
reviews = pd.read_csv('./../data/yelp_reviews_subset.csv')
reviews.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,wek3rf2MRDhyCBRR9iiGlA,vjY9m43zBdUDrmTfUMJ75g,W4ZEKkva9HpAdZG88juwyQ,5.0,0,0,0,This is a really great restaurant with awesome...,2019-12-06 04:19:05
1,szh4wJ2VjLHOOhsT1V7Dfg,qIFbnyVpwAzywSKQbSSP-w,SQX3MkZ2ve5N1guTtAq6eg,5.0,0,0,0,We just closed on our home today and we couldn...,2019-12-27 00:20:39
2,t83PSg9MrLfBahKIE8uY5w,w6z5D4f608RR5JmkT9QUuw,sGLDJ4V4ezZZemFHCa0z_g,2.0,0,0,0,Well we wanted to try a new place in Sparks. I...,2019-12-23 02:50:59
